In [1]:
import tensorflow as tf
import numpy as np
import time

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
VGG_MEAN = [123.68, 116.779, 103.939] # [R, G, B]

class vgg16:
    def __init__(self, vgg16_npy_path=None):
        """
        load pre-trained weights from path
        :param vgg16_npy_path: file path of vgg16 pre-trained weights
        """
        if vgg16_npy_path is None:
            path = inspect.getfile(Vgg16)
            path = os.path.abspath(os.path.join(path, os.pardir))
            path = os.path.join(path, "vgg16_weights.npz")
            vgg16_npy_path = path
            print(path)
        
        self.data_dict = np.load(vgg16_npy_path)
        print("npy file loaded")
        
        self.B, self.H, self.W, self.C = 32, 32, 32, 3
        self.classes = 10
        
        # dictionary for operation
        self.prob_dict = {}
        self.loss_dict = {}
        self.accu_dict = {}
        
        self.x = tf.placeholder(tf.float32, [self.B, self.H, self.W, self.C])
        self.y = tf.placeholder(tf.float32, [self.B, self.classes])

    def build(self, dp, prof_type):
        """
        load variable from npy to build the VGG
        :param rgb: rgb image [batch, height, width, 3] values scaled [0, 1]
        """
        self.dp = dp 
        print("Will optimize at DP=", self.dp)
        start_time = time.time()
        print("build model started")
        rgb_scaled = self.x * 255.0

        # normalization
        red, green, blue = tf.split(axis=3, num_or_size_splits=3, value=rgb_scaled)
        assert   red.get_shape().as_list()[1:] == [self.H, self.W, 1]
        assert green.get_shape().as_list()[1:] == [self.H, self.W, 1]
        assert  blue.get_shape().as_list()[1:] == [self.H, self.W, 1]
        self.x = tf.concat(axis=3, values=[
              red - VGG_MEAN[0],
            green - VGG_MEAN[1],
             blue - VGG_MEAN[2],
        ])
        assert self.x.get_shape().as_list()[1:] == [self.H, self.W, self.C]
        
        with tf.variable_scope("vgg16"):
        
            self.conv1_1_W, self.conv1_1_b = self.get_conv_filter("conv1_1"), self.get_bias("conv1_1")
            self.conv1_2_W, self.conv1_2_b = self.get_conv_filter("conv1_2"), self.get_bias("conv1_2")

            self.conv2_1_W, self.conv2_1_b = self.get_conv_filter("conv2_1"), self.get_bias("conv2_1")
            self.conv2_2_W, self.conv2_2_b = self.get_conv_filter("conv2_2"), self.get_bias("conv2_2")

            self.conv3_1_W, self.conv3_1_b = self.get_conv_filter("conv3_1"), self.get_bias("conv3_1")
            self.conv3_2_W, self.conv3_2_b = self.get_conv_filter("conv3_2"), self.get_bias("conv3_2")
            self.conv3_3_W, self.conv3_3_b = self.get_conv_filter("conv3_3"), self.get_bias("conv3_3")

            self.conv4_1_W, self.conv4_1_b = self.get_conv_filter("conv4_1"), self.get_bias("conv4_1")
            self.conv4_2_W, self.conv4_2_b = self.get_conv_filter("conv4_2"), self.get_bias("conv4_2")
            self.conv4_3_W, self.conv4_3_b = self.get_conv_filter("conv4_3"), self.get_bias("conv4_3")

            self.conv5_1_W, self.conv5_1_b = self.get_conv_filter("conv5_1"), self.get_bias("conv5_1")
            self.conv5_2_W, self.conv5_2_b = self.get_conv_filter("conv5_2"), self.get_bias("conv5_2")
            self.conv5_3_W, self.conv5_3_b = self.get_conv_filter("conv5_3"), self.get_bias("conv5_3")

            self.fc_1_W = tf.get_variable(name="fc_1_W", shape=(512, 512), initializer=tf.truncated_normal_initializer(mean=0, stddev=0.1), dtype=tf.float32)
            self.fc_1_b = tf.get_variable(name="fc_1_b", shape=(512), initializer=tf.truncated_normal_initializer(mean=0, stddev=0.1), dtype=tf.float32)

            self.fc_2_W = tf.get_variable(name="fc_2_W", shape=(512, 512), initializer=tf.truncated_normal_initializer(mean=0, stddev=0.1), dtype=tf.float32)
            self.fc_2_b = tf.get_variable(name="fc_2_b", shape=(512), initializer=tf.truncated_normal_initializer(mean=0, stddev=0.1), dtype=tf.float32)

            self.fc_3_W = tf.get_variable(name="fc_3_W", shape=(512, 10), initializer=tf.truncated_normal_initializer(mean=0, stddev=0.1), dtype=tf.float32)
            self.fc_3_b = tf.get_variable(name="fc_3_b", shape=(10), initializer=tf.truncated_normal_initializer(mean=0, stddev=0.1), dtype=tf.float32)

        
        # for loop for dp, assign prof_type
        for dp_i in dp:
            with tf.name_scope(str(int(dp_i*100))):
                conv1_1 = self.idp_conv_layer( self.x, "conv1_1", dp_i, prof_type, gamma_trainable=False)
                conv1_2 = self.idp_conv_layer(conv1_1, "conv1_2", dp_i, prof_type, gamma_trainable=False)
                pool1 = self.max_pool(conv1_2, 'pool1')

                conv2_1 = self.idp_conv_layer(  pool1, "conv2_1", dp_i, prof_type, gamma_trainable=False)
                conv2_2 = self.idp_conv_layer(conv2_1, "conv2_2", dp_i, prof_type, gamma_trainable=False)
                pool2 = self.max_pool(conv2_2, 'pool2')

                conv3_1 = self.idp_conv_layer(  pool2, "conv3_1", dp_i, prof_type, gamma_trainable=False)
                conv3_2 = self.idp_conv_layer(conv3_1, "conv3_2", dp_i, prof_type, gamma_trainable=False)
                conv3_3 = self.idp_conv_layer(conv3_2, "conv3_3", dp_i, prof_type, gamma_trainable=False)
                pool3 = self.max_pool(conv3_3, 'pool3')

                conv4_1 = self.idp_conv_layer(  pool3, "conv4_1", dp_i, prof_type, gamma_trainable=False)
                conv4_2 = self.idp_conv_layer(conv4_1, "conv4_2", dp_i, prof_type, gamma_trainable=False)
                conv4_3 = self.idp_conv_layer(conv4_2, "conv4_3", dp_i, prof_type, gamma_trainable=False)
                pool4 = self.max_pool(conv4_3, 'pool4')

                conv5_1 = self.idp_conv_layer(  pool4, "conv5_1", dp_i, prof_type, gamma_trainable=False)
                conv5_2 = self.idp_conv_layer(conv5_1, "conv5_2", dp_i, prof_type, gamma_trainable=False)
                conv5_3 = self.idp_conv_layer(conv5_2, "conv5_3", dp_i, prof_type, gamma_trainable=False)
                pool5 = self.max_pool(conv5_3, 'pool5')

                fc_1 = self.fc_layer(pool5, 'fc_1')
                fc_1 = tf.nn.relu(fc_1)
                fc_2 = self.fc_layer(fc_1, 'fc_2')
                fc_2 = tf.nn.relu(fc_2)
                
                logits = tf.nn.bias_add(tf.matmul( fc_2, self.fc_3_W), self.fc_3_b)
                prob = tf.nn.softmax(logits, name="prob")
                
                cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=self.y)
                loss = tf.reduce_mean(cross_entropy)
                accuracy = tf.mean(tf.equal(x=tf.argmax(logits, 1), y=tf.argmax(self.y, 1)))
                
                self.prob_dict[str(int(dp_i*100))] = prob
                self.loss_dict[str(int(dp_i*100))] = loss
                self.accu_dict[str(int(dp_i*100))] = accuracy
                
        print(("build model finished: %ds" % (time.time() - start_time)))

    def avg_pool(self, bottom, name):
        return tf.nn.avg_pool(bottom, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name=name)

    def max_pool(self, bottom, name):
        return tf.nn.max_pool(bottom, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name=name)

    def idp_conv_layer(self, bottom, name, dp, prof_type, gamma_trainable = False):
        with tf.name_scope(name+str(int(dp*100))):
            with tf.variable_scope("vgg16",reuse=True):
                conv_filter = tf.get_variable(name=name+"_W")
                conv_biases = tf.get_variable(name=name+"_b")
            
            H,W,C,O = conv_filter.get_shape().as_list()
        
            # get profile
            profile = self.get_profile(O, prof_type)
            
            # create a mask determined by the dot product percentage
            n1 = int(O * dp)
            n0 = O - n1
            mask = np.append(np.ones(n1, dtype='float32'), np.zeros(n0, dtype='float32'))
            if len(profile) == len(mask):
                profile *= mask
            else:
                raise ValueError("profile and mask must have the same shape.")

            # create a profile coefficient, gamma
            filter_profile = np.stack([profile for i in range(H*W*C)])
            filter_profile = np.reshape(filter_profile, newshape=(H, W, C, O))

            gamma_W = tf.Variable(initial_value=filter_profile, name=name+"_gamma_W_"+str(int(dp*100)), trainable=gamma_trainable)
            gamma_b = tf.Variable(initial_value=profile, name=name+"_gamma_W_"+str(int(dp*100)), trainable=gamma_trainable)

            # IDP conv2d output
            conv_filter = tf.multiply(conv_filter, gamma_W)
            conv_biases = tf.multiply(conv_biases, gamma_b)
            
            conv = tf.nn.conv2d(bottom, conv_filter, [1, 1, 1, 1], padding='SAME')
            
            conv = tf.nn.bias_add(conv, conv_biases)
            
            relu = tf.nn.relu(conv)
            return relu

    def fc_layer(self, bottom, name):
        with tf.name_scope(name):
            shape = bottom.get_shape().as_list()
            dim = 1
            for d in shape[1:]:
                dim *= d
            x = tf.reshape(bottom, [-1, dim])
            
            with tf.variable_scope("vgg16",reuse=True):
                weights = tf.get_variable(name=name+"_W")
                biases = tf.get_variable(name=name+"_b")

            # Fully connected layer. Note that the '+' operation automatically
            # broadcasts the biases.
            fc = tf.nn.bias_add(tf.matmul(x, weights), biases)
            return fc

    def get_conv_filter(self, name):
        return tf.get_variable(initializer=self.data_dict[name+"_W"], name=name+"_W")
    def get_bias(self, name):
        return tf.get_variable(initializer=self.data_dict[name+"_b"], name=name+"_b")
    def get_fc_weight(self, name):
        return tf.get_variable(initializer=self.data_dict[name+"_W"], name=name+"_W")

    def get_profile(self, C, prof_type):
        def half_exp(n, k=1, dtype='float32'):
            n_ones = int(n/2)
            n_other = n - n_ones
            return np.append(np.ones(n_ones, dtype=dtype), np.exp((1-k)*np.arange(n_other), dtype=dtype))
        if prof_type == "linear":
            profile = np.linspace(1.0,0.0, num=C, endpoint=False, dtype='float32')
        elif prof_type == "all-one":
            profile = np.ones(C, dtype='float32')
        elif prof_type == "half-exp":
            profile = half_exp(C, 2.0)
        elif prof_type == "harmonic":
            profile = np.array(1.0/(np.arange(C)+1))
        else:
            raise ValueError("prof_type must be \"all-one\", \"half-exp\", \"harmonic\" or \"linear\".")
        return profile
                

In [3]:
vgg16 = vgg16("vgg16_weights.npz")

npy file loaded


In [4]:
vgg16.build(dp=[(i+1)*0.05 for i in range(1,20)],prof_type="linear")

Will optimize at DP= [0.1, 0.15000000000000002, 0.2, 0.25, 0.30000000000000004, 0.35000000000000003, 0.4, 0.45, 0.5, 0.55, 0.6000000000000001, 0.65, 0.7000000000000001, 0.75, 0.8, 0.8500000000000001, 0.9, 0.9500000000000001, 1.0]
build model started
build model finished: 7s


In [24]:
obj = tf.add(vgg16.loss_dict['100'],vgg16.loss_dict['70'])

In [25]:
obj = tf.add(obj,vgg16.loss_dict['50'])

In [26]:
train_op = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(obj)

In [34]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(vgg16.fc_3_b))
    for i in range(1):
        sess.run(train_op, feed_dict={vgg16.x: Xtrain[:32,:,:,:],
                                         vgg16.y: Ytrain[:32,:]})
    
    perf = sess.run(vgg16.accu_dict['100'],
                       feed_dict={vgg16.x: Xtrain[:32,:,:,:],
                                  vgg16.y: Ytrain[:32,:]})
    print(sess.run(vgg16.fc_3_b))
    print("Predicted model: {a:.4f}".format(a=np.mean(perf)))

Predicted model: 0.1875


In [32]:
s1 = tf.Session()
s1.run(tf.global_variables_initializer())

In [33]:
print(s1.run(vgg16.fc_3_b))

[ 0.0172167   0.03313492  0.04479698  0.01052409 -0.0890021   0.04248823
 -0.06832712 -0.14019343 -0.166647   -0.07080839]


In [10]:
# %load utils.py
########################################################################
#
# Functions for downloading the CIFAR-10 data-set from the internet
# and loading it into memory.
#
# Implemented in Python 3.5
#
# Usage:
# 1) Set the variable data_path with the desired storage path.
# 2) Call maybe_download_and_extract() to download the data-set
#    if it is not already located in the given data_path.
# 3) Call load_class_names() to get an array of the class-names.
# 4) Call load_training_data() and load_test_data() to get
#    the images, class-numbers and one-hot encoded class-labels
#    for the training-set and test-set.
# 5) Use the returned data in your own program.
#
# Format:
# The images for the training- and test-sets are returned as 4-dim numpy
# arrays each with the shape: [image_number, height, width, channel]
# where the individual pixels are floats between 0.0 and 1.0.
#
########################################################################
#
# This file is part of the TensorFlow Tutorials available at:
#
# https://github.com/Hvass-Labs/TensorFlow-Tutorials
#
# Published under the MIT License. See the file LICENSE for details.
#
# Copyright 2016 by Magnus Erik Hvass Pedersen
#
########################################################################

import numpy as np
import pickle
import os
#import download

########################################################################

# Directory where you want to download and save the data-set.
# Set this before you start calling any of the functions below.
data_path = "/Users/chunmingchang/data/"

# URL for the data-set on the internet.
data_url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"

########################################################################
# Various constants for the size of the images.
# Use these constants in your own program.

# Width and height of each image.
img_size = 32

# Number of channels in each image, 3 channels: Red, Green, Blue.
num_channels = 3

# Length of an image when flattened to a 1-dim array.
img_size_flat = img_size * img_size * num_channels

# Number of classes.
num_classes = 10

########################################################################
# Various constants used to allocate arrays of the correct size.

# Number of files for the training-set.
_num_files_train = 5

# Number of images for each batch-file in the training-set.
_images_per_file = 10000

# Total number of images in the training-set.
# This is used to pre-allocate arrays for efficiency.
_num_images_train = _num_files_train * _images_per_file

########################################################################
# 
def _one_hot_encoded(class_numbers, num_classes=None):
    """
    Generate the One-Hot encoded class-labels from an array of integers.
    For example, if class_number=2 and num_classes=4 then
    the one-hot encoded label is the float array: [0. 0. 1. 0.]
    :param class_numbers:
        Array of integers with class-numbers.
        Assume the integers are from zero to num_classes-1 inclusive.
    :param num_classes:
        Number of classes. If None then use max(class_numbers)+1.
    :return:
        2-dim array of shape: [len(class_numbers), num_classes]
    """

    # Find the number of classes if None is provided.
    # Assumes the lowest class-number is zero.
    if num_classes is None:
        num_classes = np.max(class_numbers) + 1

    return np.eye(num_classes, dtype=float)[class_numbers]





########################################################################
# Private functions for downloading, unpacking and loading data-files.

def _get_file_path(filename=""):
    """
    Return the full path of a data-file for the data-set.

    If filename=="" then return the directory of the files.
    """

    return os.path.join(data_path, "cifar-10-batches-py/", filename)


def _unpickle(filename):
    """
    Unpickle the given file and return the data.

    Note that the appropriate dir-name is prepended the filename.
    """

    # Create full path for the file.
    file_path = _get_file_path(filename)

    print("Loading data: " + file_path)

    with open(file_path, mode='rb') as file:
        # In Python 3.X it is important to set the encoding,
        # otherwise an exception is raised here.
        data = pickle.load(file, encoding='bytes')

    return data


def _convert_images(raw):
    """
    Convert images from the CIFAR-10 format and
    return a 4-dim array with shape: [image_number, height, width, channel]
    where the pixels are floats between 0.0 and 1.0.
    """

    # Convert the raw images from the data-files to floating-points.
    raw_float = np.array(raw, dtype=float) / 255.0

    # Reshape the array to 4-dimensions.
    images = raw_float.reshape([-1, num_channels, img_size, img_size])

    # Reorder the indices of the array.
    images = images.transpose([0, 2, 3, 1])

    return images


def _load_data(filename):
    """
    Load a pickled data-file from the CIFAR-10 data-set
    and return the converted images (see above) and the class-number
    for each image.
    """

    # Load the pickled data-file.
    data = _unpickle(filename)

    # Get the raw images.
    raw_images = data[b'data']

    # Get the class-numbers for each image. Convert to numpy-array.
    cls = np.array(data[b'labels'])

    # Convert the images.
    images = _convert_images(raw_images)

    return images, cls


########################################################################
# Public functions that you may call to download the data-set from
# the internet and load the data into memory.


# def maybe_download_and_extract():
#     """
#     Download and extract the CIFAR-10 data-set if it doesn't already exist
#     in data_path (set this variable first to the desired path).
#     """

#     download.maybe_download_and_extract(url=data_url, download_dir=data_path)


def load_class_names():
    """
    Load the names for the classes in the CIFAR-10 data-set.

    Returns a list with the names. Example: names[3] is the name
    associated with class-number 3.
    """

    # Load the class-names from the pickled file.
    raw = _unpickle(filename="batches.meta")[b'label_names']

    # Convert from binary strings.
    names = [x.decode('utf-8') for x in raw]

    return names


def load_training_data():
    """
    Load all the training-data for the CIFAR-10 data-set.

    The data-set is split into 5 data-files which are merged here.

    Returns the images, class-numbers and one-hot encoded class-labels.
    """

    # Pre-allocate the arrays for the images and class-numbers for efficiency.
    images = np.zeros(shape=[_num_images_train, img_size, img_size, num_channels], dtype=float)
    cls = np.zeros(shape=[_num_images_train], dtype=int)

    # Begin-index for the current batch.
    begin = 0

    # For each data-file.
    for i in range(_num_files_train):
        # Load the images and class-numbers from the data-file.
        images_batch, cls_batch = _load_data(filename="data_batch_" + str(i + 1))

        # Number of images in this batch.
        num_images = len(images_batch)

        # End-index for the current batch.
        end = begin + num_images

        # Store the images into the array.
        images[begin:end, :] = images_batch

        # Store the class-numbers into the array.
        cls[begin:end] = cls_batch

        # The begin-index for the next batch is the current end-index.
        begin = end

    return images, _one_hot_encoded(class_numbers=cls, num_classes=num_classes)


def load_test_data():
    """
    Load all the test-data for the CIFAR-10 data-set.

    Returns the images, class-numbers and one-hot encoded class-labels.
    """

    images, cls = _load_data(filename="test_batch")

    return images, _one_hot_encoded(class_numbers=cls, num_classes=num_classes)

########################################################################

In [12]:
Xtrain, Ytrain = load_test_data()

Loading data: /Users/chunmingchang/data/cifar-10-batches-py/test_batch


In [13]:
Xtrain.shape

(10000, 32, 32, 3)

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
accu100, accu70, accu50 = sess.run([vgg16.accu_dict['100'], 
                                                    vgg16.accu_dict['70'],
                                                    vgg16.accu_dict['50']],
                                                    feed_dict={vgg16.x: Xtrain[:32,:,:,:],
                                                               vgg16.y: Ytrain[:32,:]})

In [ ]:
# %load vgg16.py
from tensorflow.python.framework import ops
from tensorflow.python.ops import gen_nn_ops
from tensorflow.python.ops.gen_nn_ops import *
import os
import numpy as np
import tensorflow as tf

def my_profile(C, prof_type):
    def half_exp(n, k=1, dtype='float32'):
        n_ones = int(n/2)
        n_other = n - n_ones
        return np.append(np.ones(n_ones, dtype=dtype), np.exp((1-k)*np.arange(n_other), dtype=dtype))
    if prof_type == "linear":
        profile = np.linspace(1.0,0.0, num=C, endpoint=False, dtype='float32')
    elif prof_type == "all-one":
        profile = np.ones(C, dtype='float32')
    elif prof_type == "half-exp":
        profile = half_exp(C, 2.0)
    elif prof_type == "harmonic":
        profile = np.array(1.0/(np.arange(C)+1))
    else:
        raise ValueError("prof_type must be \"all-one\", \"half-exp\", \"harmonic\" or \"linear\".")
    return profile

def idp_conv2d(input, filter, strides, padding,
               use_cudnn_on_gpu=True, data_format='NHWC',
               name=None, prof_type=None, dp=1.0):
    with ops.name_scope(name, "idp_convolution", [input, filter]) as scope:
        if not (data_format == "NHWC" or data_format == "NCHW"):
            raise ValueError("data_format must be \"NHWC\" or \"NCHW\".")
        
        # do conv2d
        conv2d_res = gen_nn_ops.conv2d(input, filter, strides, padding,
                                       use_cudnn_on_gpu=True,
                                       data_format='NHWC',
                                       name=None)
        B,H,W,C = conv2d_res.get_shape().as_list()
        
        # get profile
        profile = my_profile(C, prof_type)
        # tensor_profile = tf.get_variable(initializer=profile,name="tensor_profile",dtype='float32')

        # create a mask determined by the dot product percentage
        n1 = int(C * dp)
        n0 = C - n1
        mask = np.append(np.ones(n1, dtype='float32'), np.zeros(n0, dtype='float32'))
        if len(profile) == len(mask):
            profile *= mask
        else:
            raise ValueError("profile and mask must have the same shape.")

        # create a profile coefficient, gamma
        conv2d_profile = np.stack([profile for i in range(B*H*W)])
        conv2d_profile = np.reshape(conv2d_profile, newshape=(B, H, W, C))
        
        gamma = tf.get_variable(initializer=conv2d_profile, name="gamma"+str(dp*100))
        
        # IDP conv2d output
        idp_conv2d_res = tf.multiply(conv2d_res, gamma, name="idp"+str(dp*100))
        
        return idp_conv2d_res


